<a href="https://colab.research.google.com/github/TianyiZhang-zzz/SupplyChain/blob/main/sc0306.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

# Given Data
products = {
    "Product": [1, 2, 3, 4],
    "Demand": [1000, 300, 100, 50],
    "Common_Ordering_Cost": [150, 150, 150, 150],
    "Specific_Ordering_Cost": [20, 25, 30, 50],
    "Unit_Cost": [50, 60, 30, 30],
    "Holding_Cost_Rate": [0.15, 0.15, 0.15, 0.15]
}

df = pd.DataFrame(products)

# Compute Total Ordering Cost for each product
df["Total_Ordering_Cost"] = df["Common_Ordering_Cost"] + df["Specific_Ordering_Cost"]

# EOQ Calculation: Q = sqrt(2DS/hC)
df["EOQ"] = np.sqrt((2 * df["Demand"] * df["Total_Ordering_Cost"]) / (df["Holding_Cost_Rate"] * df["Unit_Cost"]))

# Independent Sourcing
df["Order_Frequency_Ind"] = df["Demand"] / df["EOQ"]
df["Annual_Ordering_Cost_Ind"] = df["Order_Frequency_Ind"] * df["Total_Ordering_Cost"]
df["Annual_Holding_Cost_Ind"] = (df["EOQ"] / 2) * df["Holding_Cost_Rate"] * df["Unit_Cost"]
total_cost_independent = df["Annual_Ordering_Cost_Ind"].sum() + df["Annual_Holding_Cost_Ind"].sum()

# Same Frequency Sourcing
agg_demand = df["Demand"].sum()
agg_ordering_cost = df["Common_Ordering_Cost"].max() + df["Specific_Ordering_Cost"].sum()
agg_unit_cost = (df["Unit_Cost"] * df["Demand"]).sum() / agg_demand  # Weighted average cost

EOQ_same = np.sqrt((2 * agg_demand * agg_ordering_cost) / (df["Holding_Cost_Rate"].mean() * agg_unit_cost))
order_frequency_same = agg_demand / EOQ_same
annual_ordering_cost_same = order_frequency_same * agg_ordering_cost
annual_holding_cost_same = (EOQ_same / 2) * df["Holding_Cost_Rate"].mean() * agg_unit_cost
total_cost_same = annual_ordering_cost_same + annual_holding_cost_same

# Tailored Aggregation Strategy (Grouping)
df["Grouped_Ordering_Cost"] = df["Common_Ordering_Cost"] + df["Specific_Ordering_Cost"].mean()
df["EOQ_Grouped"] = np.sqrt((2 * df["Demand"] * df["Grouped_Ordering_Cost"]) / (df["Holding_Cost_Rate"] * df["Unit_Cost"]))
df["Order_Frequency_Grouped"] = df["Demand"] / df["EOQ_Grouped"]
df["Annual_Ordering_Cost_Grouped"] = df["Order_Frequency_Grouped"] * df["Grouped_Ordering_Cost"]
df["Annual_Holding_Cost_Grouped"] = (df["EOQ_Grouped"] / 2) * df["Holding_Cost_Rate"] * df["Unit_Cost"]
total_cost_grouped = df["Annual_Ordering_Cost_Grouped"].sum() + df["Annual_Holding_Cost_Grouped"].sum()

# Display results
print(f"Annual Operational Cost (Independent Sourcing): ${total_cost_independent:.2f}")
print(f"Annual Operational Cost (Same Frequency Sourcing): ${total_cost_same:.2f}")
print(f"Annual Operational Cost (Tailored Aggregation Strategy): ${total_cost_grouped:.2f}")


Annual Operational Cost (Independent Sourcing): $3271.48
Annual Operational Cost (Same Frequency Sourcing): $2445.66
Annual Operational Cost (Tailored Aggregation Strategy): $3327.66
